In [ ]:
import subprocess
import sys

def install_packages():
    packages = [
        "transformers>=4.35.0",
        "peft>=0.6.0",
        "trl>=0.7.0",
        "datasets>=2.14.0",
        "accelerate>=0.24.0",
        "bitsandbytes>=0.41.0",
        "torch>=2.0.0",
        "torchvision>=0.15.0",
        "Pillow>=9.0.0",
        "requests>=2.28.0",
        "beautifulsoup4>=4.11.0",
        "scikit-learn>=1.3.0",
        "matplotlib>=3.6.0",
        "seaborn>=0.12.0",
        "pandas>=1.5.0",
        "numpy>=1.21.0",
        "opencv-python>=4.5.0",
        "pydicom>=2.3.0",
        "nibabel>=4.0.0"
    ]

    for package in packages:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# install_packages()

In [ ]:
import subprocess
import sys
import os
import torch
import json
import pandas as pd
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Dict, List, Tuple, Optional, Union
import requests
from io import BytesIO
import base64
import gc
import warnings
import cv2
from pathlib import Path
import random
from datetime import datetime
warnings.filterwarnings('ignore')

# Core ML libraries
from datasets import Dataset, load_dataset, DatasetDict
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
    LlavaForConditionalGeneration,
    LlavaProcessor,
    AutoProcessor,
    Trainer,
    DataCollatorForLanguageModeling
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer
from sklearn.metrics import accuracy_score, f1_score, precision_recall_fscore_support
from sklearn.model_selection import train_test_split


In [ ]:
print("\nConfiguration Setup...")

# Model and dataset configuration
model_name = "llava-hf/llava-1.5-7b-hf"
dataset_name = "medical-vision-papers"
new_model = "Llava-Medical-Analyst-7b"

# LoRA configuration
lora_r = 64
lora_alpha = 16
lora_dropout = 0.1

# Vision-specific LoRA targets
vision_lora_targets = [
    "q_proj", "v_proj", "k_proj", "o_proj",
    "gate_proj", "up_proj", "down_proj",
    "multi_modal_projector"
]

# Quantization configuration
use_4bit = True
bnb_4bit_compute_dtype = "float16"
bnb_4bit_quant_type = "nf4"
use_nested_quant = False

# Training configuration
output_dir = "./medical_vision_results"
num_train_epochs = 2  # was 3
per_device_train_batch_size = 2  # was 1
gradient_accumulation_steps = 2  # was 4
learning_rate = 2e-4
weight_decay = 0.001
optim = "paged_adamw_32bit"
lr_scheduler_type = "cosine"
max_grad_norm = 0.3
max_steps = -1
warmup_ratio = 0.03
group_by_length = True
logging_steps = 50  # was 25
fp16 = True
bf16 = False
gradient_checkpointing = True

print(f"Model: {model_name}")
print(f"LoRA Config: r={lora_r}, alpha={lora_alpha}, dropout={lora_dropout}")
print(f"Training: {num_train_epochs} epochs, lr={learning_rate}")
print("Configuration complete!")









Configuration Setup...
Model: llava-hf/llava-1.5-7b-hf
LoRA Config: r=64, alpha=16, dropout=0.1
Training: 2 epochs, lr=0.0002
Configuration complete!


In [ ]:
class RealMedicalDatasetCreator:
    """Create real medical datasets from multiple sources"""

    def __init__(self, data_dir: str = "./medical_data"):
        self.data_dir = Path(data_dir)
        self.data_dir.mkdir(exist_ok=True)

        # Medical image datasets available publicly
        self.dataset_sources = {
            "chest_xray": {
                "url": "https://www.kaggle.com/datasets/paultimothymooney/chest-xray-pneumonia",
                "description": "Chest X-ray images for pneumonia detection"
            },
            "brain_tumor": {
                "url": "https://www.kaggle.com/datasets/ahmedhamada0/brain-tumor-detection",
                "description": "Brain MRI images for tumor detection"
            },
            "skin_cancer": {
                "url": "https://www.kaggle.com/datasets/fanconic/skin-cancer-malignant-vs-benign",
                "description": "Skin lesion images for cancer detection"
            }
        }

        # Medical text templates
        self.instruction_templates = [
            "Analyze this medical image and provide a detailed clinical assessment including key findings and recommendations.",
            "Based on this medical image, what are the primary abnormalities and their clinical significance?",
            "Interpret this medical imaging study and correlate findings with potential clinical presentations.",
            "Provide a comprehensive analysis of this medical image including differential diagnoses.",
            "Examine this medical image and describe the pathological findings with clinical implications.",
            "What diagnostic conclusions can be drawn from this medical image?",
            "Analyze the anatomical structures and abnormalities visible in this medical image.",
            "Provide a radiological interpretation of this medical image with clinical correlations."
        ]

        # Medical terminology and findings
        self.medical_findings = {
            "chest_xray": {
                "normal": ["Clear lung fields", "Normal cardiac silhouette", "No acute findings"],
                "pneumonia": ["Consolidation", "Infiltrates", "Increased opacity", "Air bronchograms"],
                "other": ["Cardiomegaly", "Pleural effusion", "Pneumothorax", "Pulmonary edema"]
            },
            "brain_tumor": {
                "normal": ["Normal brain parenchyma", "No mass effect", "Symmetric ventricles"],
                "tumor": ["Mass lesion", "Contrast enhancement", "Surrounding edema", "Mass effect"],
                "other": ["Hemorrhage", "Infarct", "Atrophy", "Hydrocephalus"]
            },
            "skin_cancer": {
                "normal": ["Normal skin texture", "Regular pigmentation", "No concerning features"],
                "malignant": ["Irregular borders", "Color variation", "Asymmetry", "Ulceration"],
                "other": ["Benign nevus", "Seborrheic keratosis", "Inflammation", "Pigmentation"]
            }
        }

    def create_real_dataset(self, num_samples: int = 1000) -> Dataset:
        """Create a comprehensive medical dataset"""

        print("Creating real medical dataset...")

        # Try to load existing Hugging Face medical datasets
        medical_datasets = self._load_huggingface_datasets()

        # If no datasets available, create synthetic but realistic data
        if not medical_datasets:
            print("Creating synthetic medical dataset with realistic clinical scenarios...")
            medical_datasets = self._create_synthetic_medical_data(num_samples)

        # Format for training
        formatted_data = self._format_for_training(medical_datasets)

        print(f"Created {len(formatted_data)} training examples")
        return Dataset.from_list(formatted_data)

    def _load_huggingface_datasets(self) -> List[Dict]:
        """Load medical datasets from Hugging Face"""
        datasets_loaded = []

        try:
            # Try to load medical datasets
            print("Attempting to load medical datasets from Hugging Face...")

            # Load medical image datasets if available
            medical_dataset_names = [
                "keremberke/chest-xray-classification",
            ]

            for dataset_name in medical_dataset_names:
                try:
                    print(f"Loading {dataset_name}...")
                    dataset = load_dataset(dataset_name, split="train[:100]")  # Load small sample
                    datasets_loaded.extend(list(dataset))
                    print(f"Successfully loaded {len(dataset)} samples from {dataset_name}")
                except Exception as e:
                    print(f"Could not load {dataset_name}: {e}")
                    continue

        except Exception as e:
            print(f"Error loading datasets: {e}")

        return datasets_loaded

    def _create_synthetic_medical_data(self, num_samples: int) -> List[Dict]:
        """Create synthetic but realistic medical data"""

        medical_cases = []

        # Chest X-ray cases
        chest_cases = self._generate_chest_xray_cases(num_samples // 3)
        medical_cases.extend(chest_cases)

        # Brain MRI cases
        brain_cases = self._generate_brain_mri_cases(num_samples // 3)
        medical_cases.extend(brain_cases)

        # Skin lesion cases
        skin_cases = self._generate_skin_lesion_cases(num_samples // 3)
        medical_cases.extend(skin_cases)

        return medical_cases

    def _generate_chest_xray_cases(self, num_cases: int) -> List[Dict]:
        """Generate realistic chest X-ray cases"""
        cases = []

        for i in range(num_cases):
            # Random case parameters
            age = random.randint(20, 80)
            gender = random.choice(["male", "female"])
            condition = random.choice(["normal", "pneumonia", "other"])

            # Generate clinical history
            if condition == "pneumonia":
                symptoms = ["fever", "cough", "shortness of breath", "chest pain"]
                selected_symptoms = random.sample(symptoms, random.randint(2, 4))
                clinical_history = f"{age}-year-old {gender} with {', '.join(selected_symptoms)} for {random.randint(1, 7)} days"

                findings = random.sample(self.medical_findings["chest_xray"]["pneumonia"], random.randint(2, 3))
                image_description = f"Chest X-ray showing {', '.join(findings)} in the {'bilateral' if random.random() > 0.5 else 'unilateral'} lung fields"

            elif condition == "normal":
                clinical_history = f"{age}-year-old {gender} with routine screening or follow-up examination"
                findings = random.sample(self.medical_findings["chest_xray"]["normal"], random.randint(1, 2))
                image_description = f"Chest X-ray demonstrating {', '.join(findings)}"

            else:  # other conditions
                symptoms = ["chest pain", "dyspnea", "fatigue", "palpitations"]
                selected_symptoms = random.sample(symptoms, random.randint(1, 3))
                clinical_history = f"{age}-year-old {gender} with {', '.join(selected_symptoms)}"

                findings = random.sample(self.medical_findings["chest_xray"]["other"], random.randint(1, 2))
                image_description = f"Chest X-ray revealing {', '.join(findings)}"

            cases.append({
                "image_description": image_description,
                "clinical_history": clinical_history,
                "modality": "Chest X-ray",
                "condition": condition,
                "age": age,
                "gender": gender,
                "case_id": f"chest_{i:04d}"
            })

        return cases

    def _generate_brain_mri_cases(self, num_cases: int) -> List[Dict]:
        """Generate realistic brain MRI cases"""
        cases = []

        for i in range(num_cases):
            age = random.randint(25, 75)
            gender = random.choice(["male", "female"])
            condition = random.choice(["normal", "tumor", "other"])

            if condition == "tumor":
                symptoms = ["headache", "seizures", "confusion", "weakness", "vision changes"]
                selected_symptoms = random.sample(symptoms, random.randint(2, 4))
                clinical_history = f"{age}-year-old {gender} with {', '.join(selected_symptoms)} over {random.randint(2, 12)} weeks"

                findings = random.sample(self.medical_findings["brain_tumor"]["tumor"], random.randint(2, 3))
                location = random.choice(["frontal", "parietal", "temporal", "occipital", "cerebellar"])
                image_description = f"Brain MRI showing {', '.join(findings)} in the {location} region"

            elif condition == "normal":
                clinical_history = f"{age}-year-old {gender} with routine imaging or follow-up"
                findings = random.sample(self.medical_findings["brain_tumor"]["normal"], random.randint(1, 2))
                image_description = f"Brain MRI demonstrating {', '.join(findings)}"

            else:  # other conditions
                symptoms = ["headache", "dizziness", "memory problems", "focal deficits"]
                selected_symptoms = random.sample(symptoms, random.randint(1, 3))
                clinical_history = f"{age}-year-old {gender} with {', '.join(selected_symptoms)}"

                findings = random.sample(self.medical_findings["brain_tumor"]["other"], random.randint(1, 2))
                image_description = f"Brain MRI revealing {', '.join(findings)}"

            cases.append({
                "image_description": image_description,
                "clinical_history": clinical_history,
                "modality": "Brain MRI",
                "condition": condition,
                "age": age,
                "gender": gender,
                "case_id": f"brain_{i:04d}"
            })

        return cases

    def _generate_skin_lesion_cases(self, num_cases: int) -> List[Dict]:
        """Generate realistic skin lesion cases"""
        cases = []

        for i in range(num_cases):
            age = random.randint(30, 70)
            gender = random.choice(["male", "female"])
            condition = random.choice(["normal", "malignant", "other"])

            location = random.choice(["back", "chest", "arm", "leg", "face", "neck"])

            if condition == "malignant":
                clinical_history = f"{age}-year-old {gender} with changing pigmented lesion on {location} noticed over {random.randint(2, 12)} months"
                findings = random.sample(self.medical_findings["skin_cancer"]["malignant"], random.randint(2, 3))
                image_description = f"Dermoscopic image showing lesion with {', '.join(findings)}"

            elif condition == "normal":
                clinical_history = f"{age}-year-old {gender} with routine skin examination"
                findings = random.sample(self.medical_findings["skin_cancer"]["normal"], random.randint(1, 2))
                image_description = f"Dermoscopic image demonstrating {', '.join(findings)}"

            else:  # benign conditions
                clinical_history = f"{age}-year-old {gender} with skin lesion on {location} for evaluation"
                findings = random.sample(self.medical_findings["skin_cancer"]["other"], random.randint(1, 2))
                image_description = f"Dermoscopic image showing {', '.join(findings)}"

            cases.append({
                "image_description": image_description,
                "clinical_history": clinical_history,
                "modality": "Dermoscopy",
                "condition": condition,
                "age": age,
                "gender": gender,
                "case_id": f"skin_{i:04d}"
            })

        return cases

    def _format_for_training(self, medical_cases: List[Dict]) -> List[Dict]:
        """Format medical cases for training"""
        formatted_data = []

        for case in medical_cases:
            # Generate multiple instruction-response pairs per case
            for instruction_template in random.sample(self.instruction_templates, random.randint(2, 4)):

                # Create comprehensive response
                response = self._generate_medical_response(case, instruction_template)

                # Format for training
                training_text = f"<s>[INST] {instruction_template}\n\nClinical History: {case['clinical_history']}\nImaging: {case['image_description']} [/INST] {response} </s>"

                formatted_data.append({
                    "text": training_text,
                    "case_id": case["case_id"],
                    "modality": case["modality"],
                    "condition": case["condition"],
                    "instruction": instruction_template
                })

        return formatted_data

    def _generate_medical_response(self, case: Dict, instruction: str) -> str:
        """Generate comprehensive medical response"""

        # Base response structure
        response_parts = []

        # Clinical Assessment
        response_parts.append("CLINICAL ASSESSMENT:")
        response_parts.append(f"Patient: {case['age']}-year-old {case['gender']}")
        response_parts.append(f"Imaging Modality: {case['modality']}")
        response_parts.append(f"Clinical Presentation: {case['clinical_history']}")

        # Imaging Findings
        response_parts.append("\nIMAGING FINDINGS:")
        response_parts.append(f"{case['image_description']}")

        # Interpretation based on condition
        response_parts.append("\nINTERPRETATION:")
        if case['condition'] == 'normal':
            response_parts.append("No significant abnormalities detected on imaging.")
            response_parts.append("Findings are within normal limits for patient age.")
        elif case['condition'] in ['pneumonia', 'tumor', 'malignant']:
            response_parts.append(f"Findings consistent with {case['condition']}.")
            response_parts.append("Abnormalities require clinical correlation and possible intervention.")
        else:
            response_parts.append("Findings suggest pathological process requiring further evaluation.")

        # Recommendations
        response_parts.append("\nRECOMMENDATIONS:")
        if case['condition'] == 'normal':
            response_parts.append("Routine follow-up as clinically indicated.")
        else:
            response_parts.append("Clinical correlation recommended.")
            response_parts.append("Consider additional imaging or specialist consultation if clinically indicated.")

        return "\n".join(response_parts)


In [ ]:
print("\nCreating comprehensive medical dataset...")
dataset_creator = RealMedicalDatasetCreator()
dataset = dataset_creator.create_real_dataset(num_samples=500)

print(f"Dataset created with {len(dataset)} examples")
print(f"Modalities: {set(dataset['modality'])}")
print(f"Conditions: {set(dataset['condition'])}")

# Split dataset
train_data, eval_data = train_test_split(
    list(dataset),
    test_size=0.2,
    random_state=42,
    stratify=[item['condition'] for item in dataset]
)

train_dataset = Dataset.from_list(train_data)
eval_dataset = Dataset.from_list(eval_data)

print(f"Training samples: {len(train_dataset)}")
print(f"Evaluation samples: {len(eval_dataset)}")


Creating comprehensive medical dataset...
Creating real medical dataset...
Attempting to load medical datasets from Hugging Face...
Loading keremberke/chest-xray-classification...


README.md: 0.00B [00:00, ?B/s]

chest-xray-classification.py: 0.00B [00:00, ?B/s]

Could not load keremberke/chest-xray-classification: Config name is missing.
Please pick one among the available configs: ['full', 'mini']
Example of usage:
	`load_dataset('keremberke/chest-xray-classification', 'full')`
Creating synthetic medical dataset with realistic clinical scenarios...
Created 1506 training examples
Dataset created with 1506 examples
Modalities: {'Brain MRI', 'Chest X-ray', 'Dermoscopy'}
Conditions: {'malignant', 'pneumonia', 'tumor', 'normal', 'other'}
Training samples: 1204
Evaluation samples: 302


In [ ]:
print("\nLoading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# Add medical-specific tokens
medical_tokens = [
    "<MEDICAL_IMAGE>", "</MEDICAL_IMAGE>",
    "<CLINICAL_HISTORY>", "</CLINICAL_HISTORY>",
    "<IMAGING_FINDINGS>", "</IMAGING_FINDINGS>",
    "<DIAGNOSIS>", "</DIAGNOSIS>",
    "<TREATMENT>", "</TREATMENT>",
    "<NORMAL>", "<ABNORMAL>", "<URGENT>"
]
tokenizer.add_tokens(medical_tokens)
print(f"Added {len(medical_tokens)} medical-specific tokens")

# Tokenization function
def tokenize_medical_data(examples):
    """Tokenize medical training examples"""
    texts = examples["text"]

    tokenized = tokenizer(
        texts,
        truncation=True,
        padding="max_length",
        max_length=512
    )

    # Set labels for causal language modeling
    tokenized["labels"] = tokenized["input_ids"].copy()

    return tokenized





Loading tokenizer...


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/41.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/552 [00:00<?, ?B/s]

chat_template.jinja:   0%|          | 0.00/674 [00:00<?, ?B/s]

Added 13 medical-specific tokens


In [ ]:
# Tokenizing datasets
print("Tokenizing datasets...")
tokenized_train = train_dataset.map(tokenize_medical_data, batched=True, remove_columns=train_dataset.column_names)
tokenized_eval = eval_dataset.map(tokenize_medical_data, batched=True, remove_columns=eval_dataset.column_names)
print("Dataset tokenization complete!")

Tokenizing datasets...


Map:   0%|          | 0/1204 [00:00<?, ? examples/s]

Map:   0%|          | 0/302 [00:00<?, ? examples/s]

Dataset tokenization complete!


In [ ]:
print("\nLoading vision-language model...")

compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Load model
model = LlavaForConditionalGeneration.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=compute_dtype,
    trust_remote_code=True
)

# Resize embeddings for new tokens
model.resize_token_embeddings(len(tokenizer))
model.config.use_cache = False
model.config.pretraining_tp = 1

print("Model loaded successfully!")



Loading vision-language model...


config.json:   0%|          | 0.00/950 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.18G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/141 [00:00<?, ?B/s]

Model loaded successfully!


In [ ]:
print("\nPreparing model for LoRA training...")
model = prepare_model_for_kbit_training(model, use_gradient_checkpointing=True)

# Automatically find valid target modules (only torch.nn.Linear layers)
valid_lora_targets = [
    name for name, module in model.named_modules()
    if isinstance(module, torch.nn.Linear)
]

# LoRA configuration
peft_config = LoraConfig(
    r=lora_r,
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=valid_lora_targets,
)

# Apply LoRA
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

# Training arguments
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=2,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=2,
    learning_rate=2e-4,
    weight_decay=weight_decay,
    optim=optim,
    lr_scheduler_type=lr_scheduler_type,
    max_grad_norm=max_grad_norm,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    logging_steps=50,
    fp16=fp16,
    bf16=bf16,
    eval_strategy="steps",
    eval_steps=100,
    save_steps=200,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    report_to="none",
    gradient_checkpointing=gradient_checkpointing,
    dataloader_pin_memory=False,
    remove_unused_columns=False,
)


print("Training configuration set!")

# Initialize trainer
print("\nInitializing trainer...")

trainer = SFTTrainer(
    model=model,
    args=training_arguments,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
    processing_class=tokenizer,
)

print("Trainer initialized successfully!")



Preparing model for LoRA training...
trainable params: 191,382,464 || all params: 7,254,408,128 || trainable%: 2.6382
Training configuration set!

Initializing trainer...


Truncating train dataset:   0%|          | 0/1204 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/302 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Trainer initialized successfully!


In [ ]:
print("Trainer initialized successfully!")

# Training
print("\nStarting model training...")
print("Training Progress:")

# Clear cache before training
torch.cuda.empty_cache()
gc.collect()

# Train the model
trainer.train()

print("Training completed!")

Trainer initialized successfully!

Starting model training...
Training Progress:


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss,Validation Loss
100,0.048300,0.037782
200,0.034900,0.034629
300,0.033200,0.033103
400,0.033100,0.032373
500,0.030800,0.031682
600,0.031300,0.031480


Training completed!
